In [ ]:
import undetected_chromedriver as webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException,StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd 
import keyboard
import time


options = webdriver.ChromeOptions()
profile = r"C:\Users\asus\AppData\Local\Google\Chrome\User Data\Default"

# Add arguments to ChromeOptions
options.add_argument(f"--user-data-dir={profile}") 


In [ ]:
driver = webdriver.Chrome(options=options)
#driver.get("https://twitter.com/BhajanlalBjp")
driver.get("https://twitter.com/KumariDiya")
time.sleep(8)



In [ ]:
current_scroll_distance = 0
wait_time = 3
no_of_tweets = 20
wait = WebDriverWait(driver, wait_time)
tweet_set = set()
def convert_views(view):
    if any(suffix in view for suffix in ['K', 'k', 'M', 'm', 'B', 'b']):
        k = view.replace('K', '/n1000').replace('k', '/n1000').replace('M', '/n1000000').replace('m', '/n1000000').replace('B', '/n1000000000').replace('b', '/n1000000000')
        fd = float(k.split('/n')[0])
        m = int(k.split('/n')[1])
        value = float(fd * m)
    else:
        value = int(view)
    return value

In [ ]:
while len(tweet_set)<no_of_tweets:
    try:
        driver.execute_script("window.scrollBy(0, arguments[0]);", 500)  # Scroll by 200 pixels
        current_scroll_distance += 500
        time.sleep(8)
        
        if keyboard.is_pressed('q'):
            break

        info = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='css-175oi2r']/article[@data-testid='tweet']")))
        print(f'tweet_set_len{len(tweet_set)}')
        for post_box in info:
            try:
                post = post_box.find_element(By.XPATH, ".//div[@data-testid='tweetText']/span[1]").text
            except NoSuchElementException:
                post = 'No post found'

            try:
                comment = post_box.find_element(By.XPATH, ".//div[@role='group']").text
            except NoSuchElementException:
                comment = 'No comment found'

            try:
                dateS = post_box.find_element(By.XPATH, ".//a/time").get_attribute("datetime")
            except NoSuchElementException:
                dateS = 'No date found'

            tweet_set.add((post, dateS, comment))


    except Exception as e:
        print("An error occurred:", str(e))

unique_tweets = list(tweet_set)

# Create DataFrame
df = pd.DataFrame(unique_tweets, columns=['Post', 'Date', 'Comment'])
df['Date'] = pd.to_datetime(df['Date'])
df[['reply', 'reposts', 'likes', 'views']] = df['Comment'].str.split('\n', expand=True)
df['views'] = df['views'].apply(convert_views)
df['reposts'] = df['reposts'].apply(convert_views)
df['reply'] = df['reply'].apply(convert_views)
df['likes'] = df['likes'].apply(convert_views)
df['Date'] = df['Date'].dt.tz_localize(None)
df.drop(columns=['Comment'], inplace=True)
df.to_excel(f"diya_kumari_tweets_{no_of_tweets}.xlsx")

JUST FOR DATAFRAME TEST IS BELOW

In [ ]:
unique_tweets = list(tweet_set)
df = pd.DataFrame(unique_tweets, columns=['Post', 'Date', 'Comment'])
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.tz_localize(None)
df[['reply', 'reposts', 'likes', 'views']] = df['Comment'].str.split('\n', expand=True)
